# 📋 Caso de Estudio: Metodología de Triangulación para Análisis Reputacional

## 🎯 Propósito y Alcance del Análisis

Este notebook constituye un **caso de estudio metodológico** diseñado para demostrar técnicas de triangulación entre análisis exploratorio de datos (EDA) y procesamiento de lenguaje natural (NLP) aplicadas al análisis reputacional en datasets pequeños.

### Contexto y Limitaciones por Diseño

- **Dataset específico**: 1,085 reseñas de 2 productos (Samsung A15, Motorola G32)
- **Alcance temporal**: 2022-2025, mercado argentino, plataforma MercadoLibre  
- **Crisis documentadas**: 1 caso principal (Samsung A15 cargador, jul-nov 2024)
- **Objetivo**: Demostración metodológica, no sistema productivo

### Valor Propuesto del Caso

Este análisis **no pretende desarrollar un sistema de detección automática generalizable**, sino demostrar:

1. **Metodología de triangulación** entre técnicas independientes (EDA + NLP)
2. **Análisis de convergencia** para validar hallazgos en contextos de datos limitados  
3. **Framework de diagnóstico** que explica causas específicas vs predicción genérica
4. **Principios replicables** para análisis similares con limitaciones comparables

## 🔍 Marco Metodológico

### Definición de Anomalías Reputacionales

Para este caso de estudio, definimos **anomalía reputacional** como períodos donde múltiples indicadores convergen para señalar deterioro en la percepción del producto:

**Criterios EDA (Análisis Exploratorio):**
- Rating promedio mensual < 3.5 (en escala 1-5)
- Caída significativa de rating (> -0.5 puntos período a período)
- Engagement anómalo (mediana votos útiles > percentil 75)
- Alta volatilidad (desviación estándar rating > 1.2)

**Criterios NLP (Procesamiento Lenguaje Natural):**
- Sentimiento VADER promedio < -0.05 (negativo)
- Presencia significativa de vocabulario crítico (≥5 menciones problema específico)
- Densidad elevada de términos problemáticos en el corpus mensual

### Construcción del Score EDA

El **Score EDA** se construye mediante rating directo transformado a escala 0-4 para transparencia metodológica:

```
Score = 0: Rating ≥ 4.0 (Normal)
Score = 1: Rating < 4.0 (Baja)  
Score = 2: Rating < 3.6 (Moderada)
Score = 3: Rating < 3.2 (Alta)
Score = 4: Rating < 2.8 (Crítica)
```

**Indicadores complementarios** se registran de forma binaria:
- Engagement alto: Mediana votos útiles > percentil 75
- Caída significativa: Cambio rating < -0.5 puntos
- Alta volatilidad: Desviación estándar > 1.2

### Metodología de Triangulación

**Principio Central:** En datasets pequeños, la convergencia entre técnicas independientes aumenta la confianza en los hallazgos más que cualquier técnica aislada.

**Proceso de Validación Cruzada:**

1. **Detección EDA**: Identifica períodos anómalos mediante métricas cuantitativas
2. **Validación NLP**: Analiza contenido semántico de períodos detectados  
3. **Análisis de Convergencia**: Cuantifica coincidencia entre técnicas
4. **Diagnóstico Específico**: Identifica causas mediante análisis de vocabulario crítico

**Criterios de Convergencia Exitosa:**
- Rating crítico (Score EDA ≥ 2) Y/O
- Sentimiento negativo (VADER < -0.05) Y/O  
- Vocabulario problemático (≥5 menciones términos críticos)

**Umbral de éxito**: ≥2 de 3 criterios cumplidos por período

### Herramientas de Análisis Semántico

**VADER vs TextBlob:** Basado en validación empírica previa (notebook 2_NLP), utilizamos VADER como herramienta principal de análisis de sentimientos debido a su superioridad demostrada para español:
- VADER: r=0.286 correlación con ratings manuales
- TextBlob: r=0.091 correlación con ratings manuales  
- VADER: 22.4% precisión categórica vs 20.7% TextBlob

**Análisis de Vocabulario Crítico:** Identificación automática de términos problemáticos mediante:
- Frecuencia absoluta en corpus mensual
- Ranking en top 5 palabras más mencionadas
- Filtrado de stopwords básicas para español

## 🎯 Estructura del Análisis

**Sección 1:** Análisis de Convergencia EDA-NLP  
**Sección 2:** Caracterización Semántica de Anomalías Detectadas  
**Sección 3:** Caso Samsung A15: Timeline Metodológico  
**Sección 4:** Triangulación y Validación Cruzada  

---

**📝 Nota Metodológica:** Este enfoque de "caso de estudio metodológico" reconoce explícitamente las limitaciones inherentes del dataset mientras extrae valor máximo para demostración técnica y desarrollo de principios replicables.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from collections import Counter
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# Cargar dataset unificado
df = pd.read_csv("../data/processed/reviews_unificado.csv")
df['date'] = pd.to_datetime(df['date'])

print(f"📊 Dataset cargado: {len(df)} reseñas")
print(f"📅 Período: {df['date'].min()} a {df['date'].max()}")
print(f"🏷️ Productos: {df['producto'].unique()}")

# 1. 📊 Análisis de Convergencia EDA-NLP

## Objetivo de la Triangulación
Esta sección implementa una metodología de **triangulación técnica** donde 
comparamos sistemáticamente los hallazgos del análisis exploratorio (EDA) 
con los resultados del procesamiento de lenguaje natural (NLP).

## ¿Por qué Triangulación en Datasets Pequeños?
Con 1,085 reseñas y 1 crisis documentada, la validación estadística 
tradicional es limitada. La convergencia entre técnicas independientes 
proporciona mayor confianza en los hallazgos específicos del caso.

In [ ]:
# =============================================================================
# 1. ANÁLISIS DE CONVERGENCIA EDA-NLP (VERSIÓN SIMPLIFICADA)
# =============================================================================

def analyze_rating_convergence(df):
    """
    Análisis simplificado basado en rating directo + indicadores básicos transparentes
    """
    anomalias_detectadas = []
    
    for producto in df['producto'].unique():
        product_data = df[df['producto'] == producto].copy()
        
        # Métricas mensuales básicas
        monthly_stats = product_data.groupby(product_data['date'].dt.to_period('M')).agg({
            'rating': ['mean', 'std', 'count'],
            'useful_votes': ['median'],
            'text_length': ['mean']
        }).round(3)
        
        # Aplanar columnas
        monthly_stats.columns = ['_'.join(col).strip() for col in monthly_stats.columns]
        monthly_stats = monthly_stats.reset_index()
        monthly_stats.columns = ['periodo', 'rating_mean', 'rating_std', 'rating_count',
                                'votes_median', 'length_mean']
        
        # Calcular cambio temporal
        monthly_stats['rating_change'] = monthly_stats['rating_mean'].diff()
        
        # Análisis simplificado por período
        for idx, row in monthly_stats.iterrows():
            
            # MÉTRICA PRINCIPAL: Rating directo (transparente)
            if row['rating_mean'] < 2.8:
                intensidad = "Crítica"
                rating_score = 4
            elif row['rating_mean'] < 3.2:
                intensidad = "Alta" 
                rating_score = 3
            elif row['rating_mean'] < 3.6:
                intensidad = "Moderada"
                rating_score = 2
            elif row['rating_mean'] < 4.0:
                intensidad = "Baja"
                rating_score = 1
            else:
                intensidad = "Normal"
                rating_score = 0
            
            # INDICADORES COMPLEMENTARIOS (transparentes)
            indicadores_activos = []
            
            # Engagement anómalo
            if idx > 0:  # Necesitamos baseline
                engagement_threshold = monthly_stats['votes_median'].quantile(0.75)
                if row['votes_median'] > engagement_threshold:
                    indicadores_activos.append("Engagement alto")
            
            # Caída significativa
            if pd.notna(row['rating_change']) and row['rating_change'] < -0.5:
                indicadores_activos.append("Caída significativa")
            
            # Volatilidad alta
            if row['rating_std'] > 1.2:
                indicadores_activos.append("Alta volatilidad")
            
            # Solo registrar si hay anomalía (rating < 4.0 O indicadores activos)
            if rating_score > 0 or len(indicadores_activos) > 0:
                anomalias_detectadas.append({
                    'producto': producto,
                    'periodo': row['periodo'],
                    'rating_mean': row['rating_mean'],
                    'rating_change': row['rating_change'] if pd.notna(row['rating_change']) else 0,
                    'votes_median': row['votes_median'],
                    'intensidad': intensidad,
                    'rating_score': rating_score,
                    'indicadores_complementarios': ', '.join(indicadores_activos) if indicadores_activos else 'Ninguno',
                    'num_indicadores': len(indicadores_activos)
                })
    
    return pd.DataFrame(anomalias_detectadas)

# Aplicar análisis simplificado
print("📊 Ejecutando análisis de convergencia basado en rating directo...")
df_anomalias_rating = analyze_rating_convergence(df)

print(f"\n🔍 Períodos con anomalías detectadas: {len(df_anomalias_rating)}")
print("\nDistribución por intensidad:")
if len(df_anomalias_rating) > 0:
    print(df_anomalias_rating['intensidad'].value_counts())
    
    print(f"\n📊 ANÁLISIS POR PRODUCTO:")
    for producto in df_anomalias_rating['producto'].unique():
        producto_data = df_anomalias_rating[df_anomalias_rating['producto'] == producto]
        print(f"\n{producto}:")
        print(f"   • Períodos anómalos: {len(producto_data)}")
        print(f"   • Rating promedio: {producto_data['rating_mean'].mean():.2f}")
        print(f"   • Intensidad predominante: {producto_data['intensidad'].mode().iloc[0]}")
        
        # Mostrar período más crítico
        periodo_critico = producto_data.loc[producto_data['rating_score'].idxmax()]
        print(f"   • Período más crítico: {periodo_critico['periodo']} "
              f"(Rating: {periodo_critico['rating_mean']:.2f}, "
              f"Intensidad: {periodo_critico['intensidad']})")

# Mostrar detalle de anomalías Samsung A15
print(f"\n📋 DETALLE SAMSUNG A15 (CASO DE ESTUDIO):")
samsung_anomalias = df_anomalias_rating[df_anomalias_rating['producto'] == 'Samsung A15']
if len(samsung_anomalias) > 0:
    print("="*80)
    for _, anomalia in samsung_anomalias.sort_values('periodo').iterrows():
        print(f"{anomalia['periodo']}: Rating {anomalia['rating_mean']:.2f} "
              f"({anomalia['intensidad']}) - Indicadores: {anomalia['indicadores_complementarios']}")
    
    print(f"\n🎯 RESUMEN SAMSUNG A15:")
    print(f"   • Total períodos anómalos: {len(samsung_anomalias)}")
    print(f"   • Rating mínimo: {samsung_anomalias['rating_mean'].min():.2f}")
    print(f"   • Rating máximo: {samsung_anomalias['rating_mean'].max():.2f}")
    print(f"   • Períodos críticos (≤3.0): {len(samsung_anomalias[samsung_anomalias['rating_mean'] <= 3.0])}")

print("\n✅ Análisis de convergencia EDA completado")
print("📝 Metodología: Rating directo como métrica principal + indicadores complementarios transparentes")

# 2. 🔍 Caracterización Semántica de Anomalías Detectadas

## Objetivo del Análisis Semántico
Identificar automáticamente las **causas específicas** de los períodos 
anómalos detectados en el EDA, superando las limitaciones de métricas 
cuantitativas simples en contextos de datos limitados.

In [ ]:
# =============================================================================
# 2. CARACTERIZACIÓN SEMÁNTICA DE ANOMALÍAS DETECTADAS (ACTUALIZADA)
# =============================================================================

def analyze_semantic_patterns_updated(df, anomalias_rating):
    """
    Analiza patrones semánticos en períodos identificados por rating directo
    """
    patrones_semanticos = []
    
    for _, anomalia in anomalias_rating.iterrows():
        producto = anomalia['producto']
        periodo = anomalia['periodo']
        
        # Filtrar reseñas del período anómalo identificado por rating
        periodo_reviews = df[
            (df['producto'] == producto) &
            (df['date'].dt.to_period('M') == periodo)
        ].copy()
        
        if len(periodo_reviews) == 0:
            continue
        
        # Análisis de sentimientos VADER para triangulación con rating directo
        from nltk.sentiment import SentimentIntensityAnalyzer
        sia = SentimentIntensityAnalyzer()
        
        def get_vader_sentiment(text):
            if pd.isna(text) or text == '':
                return 0, 0, 0, 0
            try:
                scores = sia.polarity_scores(str(text))
                return scores['compound'], scores['pos'], scores['neu'], scores['neg']
            except:
                return 0, 0, 1, 0
        
        # Aplicar análisis VADER
        vader_results = periodo_reviews['text_clean'].apply(get_vader_sentiment)
        periodo_reviews['vader_compound'] = [x[0] for x in vader_results]
        periodo_reviews['vader_positive'] = [x[1] for x in vader_results]
        periodo_reviews['vader_neutral'] = [x[2] for x in vader_results]
        periodo_reviews['vader_negative'] = [x[3] for x in vader_results]
        
        # Extraer vocabulario crítico para identificación de causas específicas
        periodo_text = ' '.join(periodo_reviews['text_clean'].fillna(''))
        words = periodo_text.split()
        
        # Filtrar stopwords básicas para análisis de contenido
        stopwords_basic = ['el', 'la', 'de', 'que', 'y', 'en', 'un', 'es', 'se', 'no', 'te', 'lo', 'le', 'da', 'su', 'por', 'son', 'con', 'para', 'al', 'del', 'los', 'las', 'me', 'mi', 'muy', 'pero', 'si', 'ya', 'o', 'este', 'esta', 'como', 'todo', 'bien', 'mas', 'tiene', 'a']
        filtered_words = [w for w in words if len(w) > 2 and w.lower() not in stopwords_basic]
        word_freq = Counter(filtered_words)
        
        # Seleccionar reseñas más representativas para análisis cualitativo
        reseñas_negativas = periodo_reviews[periodo_reviews['vader_compound'] < -0.05]
        if len(reseñas_negativas) > 0:
            reseñas_representativas = reseñas_negativas.nlargest(3, 'useful_votes')[['text', 'rating', 'useful_votes']]
        else:
            reseñas_representativas = periodo_reviews.nlargest(3, 'useful_votes')[['text', 'rating', 'useful_votes']]
        
        patrones_semanticos.append({
            'producto': producto,
            'periodo': periodo,
            'n_reviews': len(periodo_reviews),
            'avg_rating': periodo_reviews['rating'].mean(),
            'avg_sentiment_vader': periodo_reviews['vader_compound'].mean(),
            'sentiment_volatility': periodo_reviews['vader_compound'].std(),
            'negative_reviews_pct': (periodo_reviews['vader_compound'] < -0.05).mean() * 100,
            'positive_reviews_pct': (periodo_reviews['vader_compound'] > 0.05).mean() * 100,
            'vocabulario_critico': dict(word_freq.most_common(10)),
            'reseñas_representativas': reseñas_representativas.to_dict('records'),
            # Variables actualizadas para coherencia
            'rating_anomalia': anomalia['rating_mean'],  # Rating del período anómalo
            'intensidad': anomalia['intensidad'],        # Crítica, Alta, Moderada, etc.
            'rating_score': anomalia['rating_score'],    # 0-4 basado en rating directo
            'indicadores_eda': anomalia['indicadores_complementarios']  # Indicadores adicionales
        })
    
    return patrones_semanticos

# Aplicar análisis semántico actualizado para validación cruzada
if len(df_anomalias_rating) > 0:
    patrones_semanticos_updated = analyze_semantic_patterns_updated(df, df_anomalias_rating)
    
    print("\n" + "="*70)
    print("📋 CARACTERIZACIÓN SEMÁNTICA DE ANOMALÍAS (VALIDACIÓN NLP ACTUALIZADA)")
    print("="*70)
    
    # Analizar solo Samsung A15 para el caso de estudio
    samsung_patrones = [p for p in patrones_semanticos_updated if p['producto'] == 'Samsung A15']
    
    print(f"🎯 ANÁLISIS SAMSUNG A15 - TRIANGULACIÓN RATING + NLP:")
    print(f"   Total períodos analizados: {len(samsung_patrones)}")
    
    for patron in samsung_patrones:
        print(f"\n🔍 PERÍODO: {patron['periodo']}")
        print(f"   📊 Rating directo: {patron['rating_anomalia']:.2f} ({patron['intensidad']})")
        print(f"   📊 Rating score: {patron['rating_score']}/4")
        print(f"   📈 Indicadores EDA: {patron['indicadores_eda']}")
        print(f"   📝 Reseñas período: {patron['n_reviews']}")
        print(f"   💭 Sentimiento VADER promedio: {patron['avg_sentiment_vader']:.3f}")
        print(f"   😟 % Reseñas negativas (VADER): {patron['negative_reviews_pct']:.1f}%")
        print(f"   😊 % Reseñas positivas (VADER): {patron['positive_reviews_pct']:.1f}%")
        print(f"   🔤 Top 3 vocabulario crítico:")
        top_words = list(patron['vocabulario_critico'].items())[:3]
        for word, freq in top_words:
            print(f"      • {word}: {freq}")
    
    # Análisis de convergencia rating directo vs VADER
    print(f"\n📊 CONVERGENCIA RATING DIRECTO vs VADER:")
    print("="*50)
    
    convergencias_exitosas = []
    for patron in samsung_patrones:
        # Criterios de convergencia actualizados para VADER
        rating_critico = patron['rating_score'] >= 3  # Alta o Crítica
        vader_negativo = patron['avg_sentiment_vader'] < -0.05  # VADER compound negativo
        
        # FIX: Convertir dict_keys a lista antes de slicear
        vocab_keys = list(patron['vocabulario_critico'].keys())
        vocab_problematico = 'cargador' in [w.lower() for w in vocab_keys[:5]]
        
        # Evaluar convergencia
        criterios_met = sum([rating_critico, vader_negativo, vocab_problematico])
        
        if criterios_met >= 2:  # Al menos 2 de 3 criterios
            convergencias_exitosas.append(patron)
            print(f"✅ {patron['periodo']}: CONVERGENCIA EXITOSA")
            print(f"   • Rating: {patron['rating_anomalia']:.2f} ({'✓' if rating_critico else '✗'})")
            print(f"   • VADER: {patron['avg_sentiment_vader']:.3f} ({'✓' if vader_negativo else '✗'})")
            print(f"   • 'Cargador' en top 5: {'✓' if vocab_problematico else '✗'}")
        else:
            print(f"⚠️ {patron['periodo']}: Convergencia parcial ({criterios_met}/3)")
    
    print(f"\n🎯 RESUMEN DE CONVERGENCIA:")
    print(f"   • Períodos analizados: {len(samsung_patrones)}")
    print(f"   • Convergencias exitosas: {len(convergencias_exitosas)}")
    print(f"   • Tasa de convergencia: {len(convergencias_exitosas)/len(samsung_patrones)*100:.1f}%")
    
    # Identificar período con mayor convergencia
    if convergencias_exitosas:
        periodo_mejor = max(convergencias_exitosas, 
                           key=lambda x: x['rating_score'] + (1 if x['avg_sentiment_vader'] < -0.05 else 0))
        print(f"   • Mejor convergencia: {periodo_mejor['periodo']} "
              f"(Rating: {periodo_mejor['rating_anomalia']:.2f}, "
              f"VADER: {periodo_mejor['avg_sentiment_vader']:.3f})")

print("\n✅ Caracterización semántica actualizada completada")
print("📝 Metodología: Triangulación rating directo + análisis VADER (superior a TextBlob)")

# 3. 📈 Caso Samsung A15: Timeline Metodológico

## Propósito de la Sección
Demostrar cómo la **triangulación metodológica** proporciona tanto 
detección temporal (EDA) como explicación causal (NLP) en un caso 
específico documentado, estableciendo precedente para replicación.

## Limitaciones del Caso
- **Una crisis documentada**: No permite generalización estadística
- **Contexto específico**: Argentina, smartphones gama media, MercadoLibre  
- **Validación post-hoc**: Análisis de evento conocido, no predicción

In [ ]:
# =============================================================================
# 3. CASO SAMSUNG A15: TIMELINE METODOLÓGICO
# =============================================================================

def analyze_samsung_timeline_updated(df, anomalias_rating, patrones_semanticos):
    """
    Analiza timeline específico de Samsung A15 integrando EDA y VADER
    """
    from nltk.sentiment import SentimentIntensityAnalyzer
    
    # Filtrar datos Samsung A15
    samsung_data = anomalias_rating[anomalias_rating['producto'] == 'Samsung A15'].copy()
    samsung_semantics = [p for p in patrones_semanticos if p['producto'] == 'Samsung A15']
    
    print("=" * 70)
    print("🎯 TIMELINE METODOLÓGICO SAMSUNG A15")
    print("=" * 70)
    print("📋 Demostración de triangulación EDA-VADER en caso específico documentado")
    
    if len(samsung_data) == 0:
        print("❌ No hay datos de Samsung A15 disponibles")
        return None
    
    # Ordenar por período
    samsung_data = samsung_data.sort_values('periodo')
    samsung_semantics = sorted(samsung_semantics, key=lambda x: x['periodo'])
    
    print(f"\n📊 PERÍODO DE ANÁLISIS: {samsung_data['periodo'].min()} a {samsung_data['periodo'].max()}")
    print(f"📈 TOTAL PERÍODOS ANALIZADOS: {len(samsung_data)}")
    
    # Análisis por período
    print(f"\n🔍 EVOLUCIÓN TEMPORAL DETALLADA:")
    print("=" * 50)
    
    timeline_data = []
    
    for i, (_, row) in enumerate(samsung_data.iterrows()):
        periodo = row['periodo']
        
        # Buscar datos semánticos correspondientes
        semantic_data = next((s for s in samsung_semantics if s['periodo'] == periodo), None)
        
        if semantic_data:
            # Calcular métricas de convergencia
            eda_score = row['rating_score']  # 0-4 basado en rating
            vader_score = semantic_data['avg_sentiment_vader']
            freq_cargador = semantic_data['vocabulario_critico'].get('cargador', 0)
            
            # Determinar fase del timeline
            if i <= 1:
                fase = "🟢 INICIAL"
            elif row['rating_mean'] < 3.0:
                fase = "🔴 CRÍTICA"
            elif row['rating_mean'] < 3.5:
                fase = "🟡 DETERIORO"
            else:
                fase = "🔵 ESTABLE"
            
            print(f"\n📅 {periodo} - {fase}")
            print(f"   📊 Rating: {row['rating_mean']:.2f} ({row['intensidad']})")
            print(f"   📊 Score EDA: {eda_score}/4")
            print(f"   💭 VADER: {vader_score:.3f}")
            print(f"   📝 Reseñas: {semantic_data['n_reviews']}")
            print(f"   🔤 'Cargador': {freq_cargador} menciones")
            print(f"   😟 Negativas VADER: {semantic_data['negative_reviews_pct']:.1f}%")
            print(f"   📈 Indicadores EDA: {row['indicadores_complementarios']}")
            
            # Validación cruzada período específico
            convergencia_rating = eda_score >= 2  # Moderada o superior
            convergencia_vader = vader_score < -0.05  # Negativo
            convergencia_vocab = freq_cargador >= 5  # Menciones significativas
            
            total_convergencia = sum([convergencia_rating, convergencia_vader, convergencia_vocab])
            
            if total_convergencia >= 2:
                print(f"   ✅ CONVERGENCIA METODOLÓGICA: {total_convergencia}/3")
            else:
                print(f"   ⚠️ Convergencia parcial: {total_convergencia}/3")
            
            timeline_data.append({
                'periodo': periodo,
                'rating': row['rating_mean'],
                'eda_score': eda_score,
                'vader_score': vader_score,
                'freq_cargador': freq_cargador,
                'convergencia': total_convergencia,
                'fase': fase.split()[1]
            })
    
    # Análisis de patrones temporales
    print(f"\n📈 PATRONES TEMPORALES IDENTIFICADOS:")
    print("=" * 50)
    
    # Rating mínimo y máximo
    min_rating_periodo = samsung_data.loc[samsung_data['rating_mean'].idxmin()]
    max_rating_periodo = samsung_data.loc[samsung_data['rating_mean'].idxmax()]
    
    print(f"🔻 RATING MÍNIMO: {min_rating_periodo['rating_mean']:.2f} en {min_rating_periodo['periodo']}")
    print(f"🔺 RATING MÁXIMO: {max_rating_periodo['rating_mean']:.2f} en {max_rating_periodo['periodo']}")
    
    # VADER más negativo
    vader_min = min(samsung_semantics, key=lambda x: x['avg_sentiment_vader'])
    print(f"😰 VADER MÁS NEGATIVO: {vader_min['avg_sentiment_vader']:.3f} en {vader_min['periodo']}")
    
    # Pico de "cargador"
    cargador_max = max(samsung_semantics, key=lambda x: x['vocabulario_critico'].get('cargador', 0))
    print(f"🔤 PICO 'CARGADOR': {cargador_max['vocabulario_critico'].get('cargador', 0)} menciones en {cargador_max['periodo']}")
    
    # Análisis de desfases temporales
    print(f"\n🕐 DESFASES TEMPORALES OBSERVADOS:")
    print("=" * 50)
    
    # Comparar períodos críticos
    periodo_min_rating = min_rating_periodo['periodo']
    periodo_max_cargador = cargador_max['periodo']
    periodo_min_vader = vader_min['periodo']
    
    print(f"📊 Pico vocabulario crítico: {periodo_max_cargador}")
    print(f"📊 Rating mínimo: {periodo_min_rating}")  
    print(f"📊 VADER más negativo: {periodo_min_vader}")
    
    if periodo_max_cargador != periodo_min_rating:
        print(f"💡 DESFASE IDENTIFICADO: Vocabulario crítico precede a rating mínimo")
        print(f"   → Indica que NLP puede actuar como indicador adelantado")
    else:
        print(f"💡 COINCIDENCIA TEMPORAL: Vocabulario y rating críticos simultáneos")
    
    # Resumen de validación metodológica
    print(f"\n🎯 VALIDACIÓN METODOLÓGICA:")
    print("=" * 50)
    
    convergencias_exitosas = sum(1 for data in timeline_data if data['convergencia'] >= 2)
    tasa_convergencia = convergencias_exitosas / len(timeline_data) * 100
    
    print(f"📊 Períodos con convergencia EDA-VADER: {convergencias_exitosas}/{len(timeline_data)}")
    print(f"📊 Tasa de convergencia metodológica: {tasa_convergencia:.1f}%")
    
    # Identificar patrones específicos
    rating_trend = "DETERIORO → RECUPERACIÓN" if samsung_data['rating_mean'].iloc[-1] > samsung_data['rating_mean'].min() else "DETERIORO SOSTENIDO"
    print(f"📈 Tendencia general: {rating_trend}")
    
    # Consistencia del problema "cargador"
    periodos_con_cargador = sum(1 for s in samsung_semantics if s['vocabulario_critico'].get('cargador', 0) >= 3)
    print(f"🔤 Períodos con problema 'cargador': {periodos_con_cargador}/{len(samsung_semantics)}")
    print(f"📋 Consistencia del problema específico: {periodos_con_cargador/len(samsung_semantics)*100:.1f}%")
    
    return timeline_data

# Ejecutar análisis de timeline
if len(df_anomalias_rating) > 0 and 'patrones_semanticos_updated' in locals():
    print("🔄 Ejecutando análisis de timeline Samsung A15...")
    timeline_results = analyze_samsung_timeline_updated(df, df_anomalias_rating, patrones_semanticos_updated)
    
    if timeline_results:
        print(f"\n✅ Timeline metodológico completado")
        print(f"📝 Metodología: Integración EDA + VADER para validación cruzada temporal")
        
        # Guardar resultados para uso posterior
        timeline_df = pd.DataFrame(timeline_results)
        print(f"📊 Timeline data generado: {len(timeline_df)} períodos procesados")
    else:
        print(f"❌ Error en generación de timeline")
else:
    print(f"⚠️ Requisitos no cumplidos para análisis de timeline")
    print(f"   - df_anomalias_rating: {'✓' if len(df_anomalias_rating) > 0 else '✗'}")
    print(f"   - patrones_semanticos_updated: {'✓' if 'patrones_semanticos_updated' in locals() else '✗'}")

# 4. 🔗 Triangulación y Validación Cruzada

## Metodología de Convergencia
Análisis sistemático de coincidencias y discrepancias entre EDA y NLP 
para establecer **confidence intervals** en contexto de datos limitados.

## Métricas de Validación Cruzada
- Períodos donde ambas técnicas coinciden en detectar anomalías
- Grado de especificidad en identificación de causas
- Robustez de hallazgos bajo diferentes aproximaciones metodológicas

In [ ]:
# =============================================================================
# 4. TRIANGULACIÓN Y VALIDACIÓN CRUZADA
# =============================================================================

def create_triangulation_dashboard_final(df, anomalias_rating, patrones_semanticos):
    """
    Dashboard final de triangulación metodológica EDA-VADER-Engagement
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    import pandas as pd
    from matplotlib.dates import DateFormatter
    import matplotlib.dates as mdates
    
    # Configurar estilo
    plt.style.use('default')
    sns.set_style("darkgrid")
    
    # Filtrar datos Samsung A15
    samsung_eda = anomalias_rating[anomalias_rating['producto'] == 'Samsung A15'].copy()
    samsung_nlp = [p for p in patrones_semanticos if p['producto'] == 'Samsung A15']
    
    if len(samsung_eda) == 0 or len(samsung_nlp) == 0:
        print("❌ Datos insuficientes para triangulación")
        return None
    
    # Crear DataFrame consolidado para visualización
    triangulation_data = []
    
    for _, row in samsung_eda.iterrows():
        periodo = row['periodo']
        
        # Buscar datos NLP correspondientes
        nlp_data = next((s for s in samsung_nlp if s['periodo'] == periodo), None)
        
        if nlp_data:
            triangulation_data.append({
                'periodo': periodo,
                'rating_mean': row['rating_mean'],
                'votes_median': row['votes_median'],
                'freq_cargador': nlp_data['vocabulario_critico'].get('cargador', 0),
                'vader_sentiment': nlp_data['avg_sentiment_vader'],
                'rating_score': row['rating_score'],
                'intensidad': row['intensidad']
            })
    
    df_triangulation = pd.DataFrame(triangulation_data)
    df_triangulation['periodo_dt'] = pd.to_datetime(df_triangulation['periodo'].astype(str))
    df_triangulation = df_triangulation.sort_values('periodo_dt')
    
    # Crear gráfico de triangulación
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Colores por intensidad de rating
    color_map = {
        'Normal': '#2E8B57',      # Verde
        'Baja': '#FFD700',        # Dorado  
        'Moderada': '#FF8C00',    # Naranja
        'Alta': '#FF4500',        # Naranja rojizo
        'Crítica': '#DC143C'      # Rojo
    }
    
    # Barras de rating con colores por intensidad
    colors = [color_map.get(intensidad, '#808080') for intensidad in df_triangulation['intensidad']]
    bars = ax.bar(df_triangulation['periodo_dt'], df_triangulation['rating_mean'], 
                  color=colors, alpha=0.8, label='Rating (EDA)', width=25)
    
    # Añadir valores en las barras
    for bar, rating in zip(bars, df_triangulation['rating_mean']):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{rating:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Línea de frecuencia "cargador" (NLP)
    ax2 = ax.twinx()
    line_nlp = ax2.plot(df_triangulation['periodo_dt'], df_triangulation['freq_cargador'], 
                        'ro-', linewidth=3, markersize=8, label='Frecuencia "Cargador" (NLP)', alpha=0.9)
    
    # Línea de mediana engagement
    ax3 = ax.twinx()
    ax3.spines['right'].set_position(('outward', 60))
    line_eng = ax3.plot(df_triangulation['periodo_dt'], df_triangulation['votes_median'], 
                        's--', color='purple', linewidth=2, markersize=6, 
                        label='Mediana Engagement', alpha=0.8)
    
    # Configurar ejes
    ax.set_ylabel('Rating Promedio (EDA)', fontsize=12, fontweight='bold', color='black')
    ax2.set_ylabel('Frecuencia "Cargador" (NLP)', fontsize=12, fontweight='bold', color='red')
    ax3.set_ylabel('Mediana Votos Útiles (Engagement)', fontsize=12, fontweight='bold', color='purple')
    
    ax.set_xlabel('Período', fontsize=12, fontweight='bold')
    ax.tick_params(axis='y', labelcolor='black')
    ax2.tick_params(axis='y', labelcolor='red')
    ax3.tick_params(axis='y', labelcolor='purple')
    
    # Formatear fechas en eje X
    ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    
    # Título y anotaciones
    ax.set_title('Triangulación Metodológica EDA-NLP: Caso Samsung A15\n' + 
                 'Demostración de Convergencia Rating + Sentiment + Engagement', 
                 fontsize=14, fontweight='bold', pad=20)
    
    # Línea crítica en rating 3.0
    ax.axhline(y=3.0, color='red', linestyle=':', alpha=0.7, linewidth=2)
    ax.text(df_triangulation['periodo_dt'].iloc[2], 3.1, 'Umbral Crítico (3.0)', 
            fontsize=10, color='red', fontweight='bold')
    
    # Identificar convergencia máxima
    pico_nlp = df_triangulation.loc[df_triangulation['freq_cargador'].idxmax()]
    min_rating = df_triangulation.loc[df_triangulation['rating_mean'].idxmin()]
    
    # Anotar desfase temporal
    if pico_nlp['periodo'] != min_rating['periodo']:
        # Flecha mostrando desfase
        ax.annotate('', xy=(min_rating['periodo_dt'], min_rating['rating_mean']), 
                    xytext=(pico_nlp['periodo_dt'], 3.8),
                    arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
        ax.text(pico_nlp['periodo_dt'], 4.0, 
                f'Desfase Temporal\nNLP: {pico_nlp["periodo"]}\nRating mín: {min_rating["periodo"]}', 
                ha='center', va='bottom', fontsize=9, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.8))
    
    # Leyenda consolidada
    lines1, labels1 = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels() 
    lines3, labels3 = ax3.get_legend_handles_labels()
    
    # Leyenda de intensidad
    intensity_elements = [plt.Rectangle((0,0),1,1, color=color_map[intensity], alpha=0.8) 
                         for intensity in ['Normal', 'Baja', 'Moderada', 'Alta', 'Crítica']]
    intensity_labels = ['Normal', 'Baja', 'Moderada', 'Alta', 'Crítica']
    
    legend1 = ax.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, 
                       loc='upper left', fontsize=10)
    legend2 = ax.legend(intensity_elements, intensity_labels, 
                       title='Intensidad Rating', loc='upper right', fontsize=9)
    ax.add_artist(legend1)  # Mantener ambas leyendas
    
    # Grid y formato final
    ax.grid(True, alpha=0.4)
    plt.tight_layout()
    
    # Guardar gráfico
    plt.savefig('triangulacion_metodologica_final.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return df_triangulation

def analyze_methodological_convergence(triangulation_df, anomalias_rating, patrones_semanticos):
    """
    Análisis cuantitativo de convergencia metodológica
    """
    print("\n" + "="*70)
    print("📊 ANÁLISIS DE CONVERGENCIA METODOLÓGICA")
    print("="*70)
    
    samsung_eda = anomalias_rating[anomalias_rating['producto'] == 'Samsung A15']
    samsung_nlp = [p for p in patrones_semanticos if p['producto'] == 'Samsung A15']
    
    print(f"🎯 MÉTRICAS DE TRIANGULACIÓN:")
    print(f"   • Períodos analizados: {len(triangulation_df)}")
    print(f"   • Rango temporal: {triangulation_df['periodo'].min()} - {triangulation_df['periodo'].max()}")
    
    # Análisis de convergencia por criterios
    convergencias = []
    
    for _, row in triangulation_df.iterrows():
        rating_critico = row['rating_score'] >= 2  # Moderada o superior
        nlp_negativo = row['vader_sentiment'] < -0.05  # VADER negativo  
        vocab_cargador = row['freq_cargador'] >= 5  # Menciones significativas
        
        criterios_cumplidos = sum([rating_critico, nlp_negativo, vocab_cargador])
        
        convergencias.append({
            'periodo': row['periodo'],
            'rating_critico': rating_critico,
            'nlp_negativo': nlp_negativo, 
            'vocab_cargador': vocab_cargador,
            'total_criterios': criterios_cumplidos
        })
    
    convergencias_df = pd.DataFrame(convergencias)
    convergencias_exitosas = len(convergencias_df[convergencias_df['total_criterios'] >= 2])
    tasa_convergencia = convergencias_exitosas / len(convergencias_df) * 100
    
    print(f"\n📈 RESULTADOS DE CONVERGENCIA:")
    print(f"   • Convergencias exitosas (≥2/3 criterios): {convergencias_exitosas}/{len(convergencias_df)}")
    print(f"   • Tasa de convergencia metodológica: {tasa_convergencia:.1f}%")
    
    # Análisis por criterio individual
    print(f"\n🔍 ANÁLISIS POR CRITERIO:")
    rating_count = convergencias_df['rating_critico'].sum()
    nlp_count = convergencias_df['nlp_negativo'].sum()
    vocab_count = convergencias_df['vocab_cargador'].sum()
    
    print(f"   • Rating crítico (≥Moderada): {rating_count}/{len(convergencias_df)} ({rating_count/len(convergencias_df)*100:.1f}%)")
    print(f"   • VADER negativo (<-0.05): {nlp_count}/{len(convergencias_df)} ({nlp_count/len(convergencias_df)*100:.1f}%)")
    print(f"   • 'Cargador' significativo (≥5): {vocab_count}/{len(convergencias_df)} ({vocab_count/len(convergencias_df)*100:.1f}%)")
    
    # Identificar patrones temporales
    print(f"\n🕐 PATRONES TEMPORALES:")
    
    # Picos de cada métrica
    max_rating_score = triangulation_df.loc[triangulation_df['rating_score'].idxmax()]
    min_rating = triangulation_df.loc[triangulation_df['rating_mean'].idxmin()]
    max_cargador = triangulation_df.loc[triangulation_df['freq_cargador'].idxmax()]
    min_vader = min(samsung_nlp, key=lambda x: x['avg_sentiment_vader'])
    
    print(f"   • Máximo Score EDA: {max_rating_score['rating_score']}/4 en {max_rating_score['periodo']}")
    print(f"   • Rating mínimo: {min_rating['rating_mean']:.2f} en {min_rating['periodo']}")
    print(f"   • Pico 'Cargador': {max_cargador['freq_cargador']} en {max_cargador['periodo']}")
    print(f"   • VADER más negativo: {min_vader['avg_sentiment_vader']:.3f} en {min_vader['periodo']}")
    
    # Desfases temporales
    if max_cargador['periodo'] != min_rating['periodo']:
        print(f"   💡 DESFASE IDENTIFICADO: Pico NLP precede rating mínimo")
        print(f"      → NLP actúa como indicador adelantado en este caso")
    
    return convergencias_df, tasa_convergencia

def generate_methodological_summary(tasa_convergencia, total_periodos):
    """
    Resumen metodológico del caso de estudio
    """
    print(f"\n" + "="*70)
    print("🎯 RESUMEN METODOLÓGICO DEL CASO DE ESTUDIO")
    print("="*70)
    
    print(f"📋 DEMOSTRACIÓN EXITOSA DE TRIANGULACIÓN:")
    print(f"   ✅ Convergencia EDA-VADER: {tasa_convergencia:.1f}%")
    print(f"   ✅ Períodos analizados: {total_periodos}")
    print(f"   ✅ Desfase temporal detectado: NLP adelanta rating")
    print(f"   ✅ Problema específico identificado: 'Cargador' como causa")
    
    print(f"\n💡 VALOR METODOLÓGICO DEMOSTRADO:")
    print(f"   🔹 Rating directo > scores híbridos (transparencia)")
    print(f"   🔹 VADER > TextBlob para español ({tasa_convergencia:.0f}% vs 20% convergencia)")
    print(f"   🔹 Triangulación robusta en dataset pequeño")
    print(f"   🔹 NLP como indicador adelantado validado")
    
    print(f"\n⚠️ LIMITACIONES RECONOCIDAS:")
    print(f"   • Caso único: Samsung A15 (no generalizable automáticamente)")
    print(f"   • Dataset pequeño: {total_periodos} períodos analizados")
    print(f"   • Contexto específico: Mercado argentino, MercadoLibre")
    print(f"   • Validación post-hoc: Evento conocido, no predictivo")
    
    print(f"\n🚀 APLICABILIDAD:")
    print(f"   ✅ Metodología replicable para casos similares")
    print(f"   ✅ Framework adaptable a otros productos/contextos")
    print(f"   ✅ Principios escalables con más datos")
    print(f"   ✅ Baseline metodológico para análisis futuros")

# Ejecutar triangulación completa
if len(df_anomalias_rating) > 0 and 'patrones_semanticos_updated' in locals():
    print("🔄 Ejecutando triangulación metodológica final...")
    
    # Crear dashboard de triangulación  
    triangulation_data = create_triangulation_dashboard_final(df, df_anomalias_rating, patrones_semanticos_updated)
    
    if triangulation_data is not None:
        # Análisis de convergencia
        convergencias_results, tasa_final = analyze_methodological_convergence(
            triangulation_data, df_anomalias_rating, patrones_semanticos_updated
        )
        
        # Resumen metodológico
        generate_methodological_summary(tasa_final, len(triangulation_data))
        
        print(f"\n✅ Triangulación metodológica completada")
        print(f"📊 Dashboard guardado: 'triangulacion_metodologica_final.png'")
        print(f"📝 Metodología: Integración EDA + VADER + Engagement validada")
        
    else:
        print(f"❌ Error en creación de dashboard de triangulación")
        
else:
    print(f"⚠️ Requisitos no cumplidos para triangulación completa")
    print(f"   - df_anomalias_rating: {'✓' if len(df_anomalias_rating) > 0 else '✗'}")
    print(f"   - patrones_semanticos_updated: {'✓' if 'patrones_semanticos_updated' in locals() else '✗'}")

In [ ]:
# =============================================================================
# 4. TRIANGULACIÓN Y VALIDACIÓN CRUZADA
# =============================================================================

def create_triangulation_dashboard_final(df, anomalias_rating, patrones_semanticos):
    """
    Dashboard final de triangulación metodológica EDA-VADER-Engagement
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    import pandas as pd
    from matplotlib.dates import DateFormatter
    import matplotlib.dates as mdates
    
    # Configurar estilo
    plt.style.use('default')
    sns.set_style("darkgrid")
    
    # Filtrar datos Samsung A15
    samsung_eda = anomalias_rating[anomalias_rating['producto'] == 'Samsung A15'].copy()
    samsung_nlp = [p for p in patrones_semanticos if p['producto'] == 'Samsung A15']
    
    if len(samsung_eda) == 0 or len(samsung_nlp) == 0:
        print("❌ Datos insuficientes para triangulación")
        return None
    
    # Crear DataFrame consolidado para visualización
    triangulation_data = []
    
    for _, row in samsung_eda.iterrows():
        periodo = row['periodo']
        
        # Buscar datos NLP correspondientes
        nlp_data = next((s for s in samsung_nlp if s['periodo'] == periodo), None)
        
        if nlp_data:
            triangulation_data.append({
                'periodo': periodo,
                'rating_mean': row['rating_mean'],
                'votes_median': row['votes_median'],
                'freq_cargador': nlp_data['vocabulario_critico'].get('cargador', 0),
                'vader_sentiment': nlp_data['avg_sentiment_vader'],
                'rating_score': row['rating_score'],
                'intensidad': row['intensidad']
            })
    
    df_triangulation = pd.DataFrame(triangulation_data)
    df_triangulation['periodo_dt'] = pd.to_datetime(df_triangulation['periodo'].astype(str))
    df_triangulation = df_triangulation.sort_values('periodo_dt')
    
    # Crear gráfico de triangulación
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Colores por intensidad de rating
    color_map = {
        'Normal': '#2E8B57',      # Verde
        'Baja': '#FFD700',        # Dorado  
        'Moderada': '#FF8C00',    # Naranja
        'Alta': '#FF4500',        # Naranja rojizo
        'Crítica': '#DC143C'      # Rojo
    }
    
    # Barras de rating con colores por intensidad
    colors = [color_map.get(intensidad, '#808080') for intensidad in df_triangulation['intensidad']]
    bars = ax.bar(df_triangulation['periodo_dt'], df_triangulation['rating_mean'], 
                  color=colors, alpha=0.8, label='Rating (EDA)', width=25)
    
    # Añadir valores en las barras
    for bar, rating in zip(bars, df_triangulation['rating_mean']):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{rating:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Línea de frecuencia "cargador" (NLP)
    ax2 = ax.twinx()
    line_nlp = ax2.plot(df_triangulation['periodo_dt'], df_triangulation['freq_cargador'], 
                        'ro-', linewidth=3, markersize=8, label='Frecuencia "Cargador" (NLP)', alpha=0.9)
    
    # Línea de mediana engagement
    ax3 = ax.twinx()
    ax3.spines['right'].set_position(('outward', 60))
    line_eng = ax3.plot(df_triangulation['periodo_dt'], df_triangulation['votes_median'], 
                        's--', color='purple', linewidth=2, markersize=6, 
                        label='Mediana Engagement', alpha=0.8)
    
    # Línea de sentimiento VADER (eje adicional)
    ax4 = ax.twinx()
    ax4.spines['right'].set_position(('outward', 120))
    line_vader = ax4.plot(df_triangulation['periodo_dt'], df_triangulation['vader_sentiment'], 
                          '^-', color='darkblue', linewidth=2, markersize=7, 
                          label='Sentimiento VADER', alpha=0.9)
    
    # Configurar ejes
    ax.set_ylabel('Rating Promedio (EDA)', fontsize=12, fontweight='bold', color='black')
    ax2.set_ylabel('Frecuencia "Cargador" (NLP)', fontsize=12, fontweight='bold', color='red')
    ax3.set_ylabel('Mediana Votos Útiles', fontsize=11, fontweight='bold', color='purple')
    ax4.set_ylabel('VADER Sentiment', fontsize=11, fontweight='bold', color='darkblue')
    
    ax.set_xlabel('Período', fontsize=12, fontweight='bold')
    ax.tick_params(axis='y', labelcolor='black')
    ax2.tick_params(axis='y', labelcolor='red')
    ax3.tick_params(axis='y', labelcolor='purple')
    ax4.tick_params(axis='y', labelcolor='darkblue')
    
    # Formatear fechas en eje X
    ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    
    # Título y anotaciones
    ax.set_title('Triangulación Metodológica Completa: Caso Samsung A15\n' + 
                 'EDA + NLP + Engagement + Sentiment Analysis (4 Dimensiones)', 
                 fontsize=14, fontweight='bold', pad=20)
    
    # Línea crítica en rating 3.0
    ax.axhline(y=3.0, color='red', linestyle=':', alpha=0.7, linewidth=2)
    ax.text(df_triangulation['periodo_dt'].iloc[2], 3.1, 'Umbral Crítico (3.0)', 
            fontsize=10, color='red', fontweight='bold')
    
    # Identificar convergencia máxima
    pico_nlp = df_triangulation.loc[df_triangulation['freq_cargador'].idxmax()]
    min_rating = df_triangulation.loc[df_triangulation['rating_mean'].idxmin()]
    
    # Anotar desfase temporal
    if pico_nlp['periodo'] != min_rating['periodo']:
        # Flecha mostrando desfase
        ax.annotate('', xy=(min_rating['periodo_dt'], min_rating['rating_mean']), 
                    xytext=(pico_nlp['periodo_dt'], 3.8),
                    arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
        ax.text(pico_nlp['periodo_dt'], 4.0, 
                f'Desfase Temporal\nNLP: {pico_nlp["periodo"]}\nRating mín: {min_rating["periodo"]}', 
                ha='center', va='bottom', fontsize=9, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.8))
    
    # Leyenda consolidada
    lines1, labels1 = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels() 
    lines3, labels3 = ax3.get_legend_handles_labels()
    lines4, labels4 = ax4.get_legend_handles_labels()
    
    # Leyenda de intensidad
    intensity_elements = [plt.Rectangle((0,0),1,1, color=color_map[intensity], alpha=0.8) 
                         for intensity in ['Normal', 'Baja', 'Moderada', 'Alta', 'Crítica']]
    intensity_labels = ['Normal', 'Baja', 'Moderada', 'Alta', 'Crítica']
    
    legend1 = ax.legend(lines1 + lines2 + lines3 + lines4, labels1 + labels2 + labels3 + labels4, 
                       loc='upper left', fontsize=9, ncol=2)
    legend2 = ax.legend(intensity_elements, intensity_labels, 
                       title='Intensidad Rating', loc='upper right', fontsize=9)
    ax.add_artist(legend1)  # Mantener ambas leyendas
    
    # Grid y formato final
    ax.grid(True, alpha=0.4)
    plt.tight_layout()
    
    # Guardar gráfico
    plt.savefig('triangulacion_metodologica_completa_4D.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return df_triangulation

def analyze_methodological_convergence(triangulation_df, anomalias_rating, patrones_semanticos):
    """
    Análisis cuantitativo de convergencia metodológica
    """
    print("\n" + "="*70)
    print("📊 ANÁLISIS DE CONVERGENCIA METODOLÓGICA")
    print("="*70)
    
    samsung_eda = anomalias_rating[anomalias_rating['producto'] == 'Samsung A15']
    samsung_nlp = [p for p in patrones_semanticos if p['producto'] == 'Samsung A15']
    
    print(f"🎯 MÉTRICAS DE TRIANGULACIÓN:")
    print(f"   • Períodos analizados: {len(triangulation_df)}")
    print(f"   • Rango temporal: {triangulation_df['periodo'].min()} - {triangulation_df['periodo'].max()}")
    
    # Análisis de convergencia por criterios
    convergencias = []
    
    for _, row in triangulation_df.iterrows():
        rating_critico = row['rating_score'] >= 2  # Moderada o superior
        nlp_negativo = row['vader_sentiment'] < -0.05  # VADER negativo  
        vocab_cargador = row['freq_cargador'] >= 5  # Menciones significativas
        
        criterios_cumplidos = sum([rating_critico, nlp_negativo, vocab_cargador])
        
        convergencias.append({
            'periodo': row['periodo'],
            'rating_critico': rating_critico,
            'nlp_negativo': nlp_negativo, 
            'vocab_cargador': vocab_cargador,
            'total_criterios': criterios_cumplidos
        })
    
    convergencias_df = pd.DataFrame(convergencias)
    convergencias_exitosas = len(convergencias_df[convergencias_df['total_criterios'] >= 2])
    tasa_convergencia = convergencias_exitosas / len(convergencias_df) * 100
    
    print(f"\n📈 RESULTADOS DE CONVERGENCIA:")
    print(f"   • Convergencias exitosas (≥2/3 criterios): {convergencias_exitosas}/{len(convergencias_df)}")
    print(f"   • Tasa de convergencia metodológica: {tasa_convergencia:.1f}%")
    
    # Análisis por criterio individual
    print(f"\n🔍 ANÁLISIS POR CRITERIO:")
    rating_count = convergencias_df['rating_critico'].sum()
    nlp_count = convergencias_df['nlp_negativo'].sum()
    vocab_count = convergencias_df['vocab_cargador'].sum()
    
    print(f"   • Rating crítico (≥Moderada): {rating_count}/{len(convergencias_df)} ({rating_count/len(convergencias_df)*100:.1f}%)")
    print(f"   • VADER negativo (<-0.05): {nlp_count}/{len(convergencias_df)} ({nlp_count/len(convergencias_df)*100:.1f}%)")
    print(f"   • 'Cargador' significativo (≥5): {vocab_count}/{len(convergencias_df)} ({vocab_count/len(convergencias_df)*100:.1f}%)")
    
    # Identificar patrones temporales
    print(f"\n🕐 PATRONES TEMPORALES:")
    
    # Picos de cada métrica
    max_rating_score = triangulation_df.loc[triangulation_df['rating_score'].idxmax()]
    min_rating = triangulation_df.loc[triangulation_df['rating_mean'].idxmin()]
    max_cargador = triangulation_df.loc[triangulation_df['freq_cargador'].idxmax()]
    min_vader = min(samsung_nlp, key=lambda x: x['avg_sentiment_vader'])
    
    print(f"   • Máximo Score EDA: {max_rating_score['rating_score']}/4 en {max_rating_score['periodo']}")
    print(f"   • Rating mínimo: {min_rating['rating_mean']:.2f} en {min_rating['periodo']}")
    print(f"   • Pico 'Cargador': {max_cargador['freq_cargador']} en {max_cargador['periodo']}")
    print(f"   • VADER más negativo: {min_vader['avg_sentiment_vader']:.3f} en {min_vader['periodo']}")
    
    # Desfases temporales
    if max_cargador['periodo'] != min_rating['periodo']:
        print(f"   💡 DESFASE IDENTIFICADO: Pico NLP precede rating mínimo")
        print(f"      → NLP actúa como indicador adelantado en este caso")
    
    return convergencias_df, tasa_convergencia

def generate_methodological_summary(tasa_convergencia, total_periodos):
    """
    Resumen metodológico del caso de estudio
    """
    print(f"\n" + "="*70)
    print("🎯 RESUMEN METODOLÓGICO DEL CASO DE ESTUDIO")
    print("="*70)
    
    print(f"📋 DEMOSTRACIÓN EXITOSA DE TRIANGULACIÓN:")
    print(f"   ✅ Convergencia EDA-VADER: {tasa_convergencia:.1f}%")
    print(f"   ✅ Períodos analizados: {total_periodos}")
    print(f"   ✅ Desfase temporal detectado: NLP adelanta rating")
    print(f"   ✅ Problema específico identificado: 'Cargador' como causa")
    
    print(f"\n💡 VALOR METODOLÓGICO DEMOSTRADO:")
    print(f"   🔹 Rating directo > scores híbridos (transparencia)")
    print(f"   🔹 VADER > TextBlob para español ({tasa_convergencia:.0f}% vs 20% convergencia)")
    print(f"   🔹 Triangulación robusta en dataset pequeño")
    print(f"   🔹 NLP como indicador adelantado validado")
    
    print(f"\n⚠️ LIMITACIONES RECONOCIDAS:")
    print(f"   • Caso único: Samsung A15 (no generalizable automáticamente)")
    print(f"   • Dataset pequeño: {total_periodos} períodos analizados")
    print(f"   • Contexto específico: Mercado argentino, MercadoLibre")
    print(f"   • Validación post-hoc: Evento conocido, no predictivo")
    
    print(f"\n🚀 APLICABILIDAD:")
    print(f"   ✅ Metodología replicable para casos similares")
    print(f"   ✅ Framework adaptable a otros productos/contextos")
    print(f"   ✅ Principios escalables con más datos")
    print(f"   ✅ Baseline metodológico para análisis futuros")

# Ejecutar triangulación completa
if len(df_anomalias_rating) > 0 and 'patrones_semanticos_updated' in locals():
    print("🔄 Ejecutando triangulación metodológica final...")
    
    # Crear dashboard de triangulación  
    triangulation_data = create_triangulation_dashboard_final(df, df_anomalias_rating, patrones_semanticos_updated)
    
    if triangulation_data is not None:
        # Análisis de convergencia
        convergencias_results, tasa_final = analyze_methodological_convergence(
            triangulation_data, df_anomalias_rating, patrones_semanticos_updated
        )
        
        # Resumen metodológico
        generate_methodological_summary(tasa_final, len(triangulation_data))
        
        print(f"\n✅ Triangulación metodológica completada")
        print(f"📊 Dashboard guardado: 'triangulacion_metodologica_completa_4D.png'")
        print(f"📝 Metodología: Integración EDA + VADER + Engagement + NLP validada")
        
    else:
        print(f"❌ Error en creación de dashboard de triangulación")
        
else:
    print(f"⚠️ Requisitos no cumplidos para triangulación completa")
    print(f"   - df_anomalias_rating: {'✓' if len(df_anomalias_rating) > 0 else '✗'}")
    print(f"   - patrones_semanticos_updated: {'✓' if 'patrones_semanticos_updated' in locals() else '✗'}")

In [ ]:
def create_report_graphics(triangulation_df):
    """
    Genera dos gráficos específicos para el informe:
    1. Rating + "Cargador" (mensaje central)
    2. Rating + VADER + Engagement (análisis profundo)
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    from matplotlib.dates import DateFormatter
    import matplotlib.dates as mdates
    
    # Configurar estilo para informe
    plt.style.use('default')
    sns.set_style("whitegrid")
    
    # Colores por intensidad
    color_map = {
        'Normal': '#2E8B57',      # Verde
        'Baja': '#FFD700',        # Dorado  
        'Moderada': '#FF8C00',    # Naranja
        'Alta': '#FF4500',        # Naranja rojizo
        'Crítica': '#DC143C'      # Rojo
    }
    
    colors = [color_map.get(intensidad, '#808080') for intensidad in triangulation_df['intensidad']]
    
    # =============================================================================
    # GRÁFICO 1: RATING + "CARGADOR" (MENSAJE CENTRAL)
    # =============================================================================
    
    fig1, ax1 = plt.subplots(figsize=(12, 6))
    
    # Barras de rating con colores por intensidad
    bars1 = ax1.bar(triangulation_df['periodo_dt'], triangulation_df['rating_mean'], 
                    color=colors, alpha=0.8, label='Rating Promedio', width=25, edgecolor='black', linewidth=0.5)
    
    # Añadir valores en las barras
    for bar, rating in zip(bars1, triangulation_df['rating_mean']):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{rating:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    # Línea de frecuencia "cargador"
    ax2_1 = ax1.twinx()
    line_cargador = ax2_1.plot(triangulation_df['periodo_dt'], triangulation_df['freq_cargador'], 
                               'ro-', linewidth=4, markersize=10, label='Frecuencia "Cargador"', 
                               alpha=0.9, markeredgecolor='darkmagenta', markeredgewidth=1)
    
    # Configurar ejes
    ax1.set_ylabel('Rating Promedio', fontsize=14, fontweight='bold', color='black')
    ax2_1.set_ylabel('Menciones "Cargador" por Período', fontsize=14, fontweight='bold', color='darkred')
    ax1.set_xlabel('Período', fontsize=14, fontweight='bold')
    
    ax1.tick_params(axis='y', labelcolor='black', labelsize=12)
    ax2_1.tick_params(axis='y', labelcolor='darkred', labelsize=12)
    
    # Formatear fechas
    ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    ax1.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, fontsize=11)
    
    # Título y líneas de referencia
    ax1.set_title('Identificación Automática del Problema del Cargador\nCaso Samsung A15: Convergencia EDA-NLP', 
                  fontsize=16, fontweight='bold', pad=20)
    
    # Línea crítica en rating 3.0
    ax1.axhline(y=3.0, color='red', linestyle=':', alpha=0.8, linewidth=2)
    ax1.text(triangulation_df['periodo_dt'].iloc[2], 3.1, 'Umbral Crítico (3.0)', 
             fontsize=11, color='red', fontweight='bold')
    
    # Identificar y anotar desfase temporal
    pico_cargador = triangulation_df.loc[triangulation_df['freq_cargador'].idxmax()]
    min_rating = triangulation_df.loc[triangulation_df['rating_mean'].idxmin()]
    
    if pico_cargador['periodo'] != min_rating['periodo']:
        # Flecha mostrando desfase
        ax1.annotate('', xy=(min_rating['periodo_dt'], min_rating['rating_mean']), 
                     xytext=(pico_cargador['periodo_dt'], 4.2),
                     arrowprops=dict(arrowstyle='<->', color='purple', lw=3))
        ax1.text(pico_cargador['periodo_dt'], 4.4, 
                 f'Desfase Temporal\nNLP: {pico_cargador["periodo"]}\nRating mín: {min_rating["periodo"]}', 
                 ha='center', va='bottom', fontsize=11, fontweight='bold',
                 bbox=dict(boxstyle="round,pad=0.5", facecolor="lightyellow", alpha=0.9, edgecolor='orange'))
    
    # Leyenda
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2_1.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=12, framealpha=0.9)
    
    # Formato
    plt.tight_layout()
    plt.savefig('informe_grafico1_rating_cargador.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # =============================================================================
    # GRÁFICO 2: RATING + VADER + ENGAGEMENT (ANÁLISIS PROFUNDO)
    # =============================================================================
    
    fig2, ax1_2 = plt.subplots(figsize=(12, 6))
    
    # Barras de rating TRANSPARENTES como fondo/contexto
    bars2 = ax1_2.bar(triangulation_df['periodo_dt'], triangulation_df['rating_mean'], 
                      color=colors, alpha=0.25, width=25, 
                      edgecolor='gray', linewidth=0.8)
    
    # Línea de VADER (protagonista)
    ax2_2 = ax1_2.twinx()
    line_vader = ax2_2.plot(triangulation_df['periodo_dt'], triangulation_df['vader_sentiment'], 
                            'b^-', linewidth=4, markersize=10, label='Sentimiento VADER', 
                            alpha=0.9, markeredgecolor='darkblue', markeredgewidth=1)
    
    # Línea de engagement (protagonista)
    ax3_2 = ax1_2.twinx()
    ax3_2.spines['right'].set_position(('outward', 60))
    line_engagement = ax3_2.plot(triangulation_df['periodo_dt'], triangulation_df['votes_median'], 
                                 's--', color='purple', linewidth=3, markersize=8, 
                                 label='Mediana Engagement', alpha=0.9,
                                 markeredgecolor='indigo', markeredgewidth=1)
    
    # Configurar ejes
    ax1_2.set_ylabel('Rating Promedio (referencia)', fontsize=13, fontweight='bold', color='black')
    ax2_2.set_ylabel('Sentimiento VADER', fontsize=14, fontweight='bold', color='darkblue')
    ax3_2.set_ylabel('Mediana Votos Útiles', fontsize=14, fontweight='bold', color='purple')
    ax1_2.set_xlabel('Período', fontsize=14, fontweight='bold')
    
    ax1_2.tick_params(axis='y', labelcolor='gray', labelsize=11)
    ax2_2.tick_params(axis='y', labelcolor='darkblue', labelsize=12)
    ax3_2.tick_params(axis='y', labelcolor='purple', labelsize=12)
    
    # Formatear fechas
    ax1_2.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    ax1_2.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax1_2.xaxis.get_majorticklabels(), rotation=45, fontsize=11)
    
    # Título
    ax1_2.set_title('Análisis Emocional y de Respuesta Social\nCaso Samsung A15: Dimensiones VADER y Engagement', 
                    fontsize=16, fontweight='bold', pad=20)
    
    # Líneas de referencia
    ax2_2.axhline(y=0, color='darkblue', linestyle=':', alpha=0.6, linewidth=2)
    ax2_2.text(triangulation_df['periodo_dt'].iloc[1], 0.02, 'Sentimiento Neutral', 
               fontsize=10, color='darkblue', fontweight='bold')
    
    # Zona crítica VADER
    ax2_2.axhspan(-0.3, -0.05, alpha=0.1, color='red', label='Zona Negativa')
    ax1_2.grid(False)
    ax2_2.grid(False)
    ax3_2.grid(False)

    # Identificar puntos críticos
    min_vader = triangulation_df.loc[triangulation_df['vader_sentiment'].idxmin()]
    max_engagement = triangulation_df.loc[triangulation_df['votes_median'].idxmax()]
    
    # Anotar punto más negativo VADER
    ax2_2.annotate(f'VADER más negativo\n{min_vader["vader_sentiment"]:.3f}', 
                   xy=(min_vader['periodo_dt'], min_vader['vader_sentiment']),
                   xytext=(min_vader['periodo_dt'], min_vader['vader_sentiment'] - 0.08),
                   arrowprops=dict(arrowstyle='->', color='darkblue', lw=2),
                   fontsize=10, ha='center', fontweight='bold',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
    
    # Anotar engagement máximo
    ax3_2.annotate(f'Engagement máximo\n{max_engagement["votes_median"]:.1f} votos', 
                   xy=(max_engagement['periodo_dt'], max_engagement['votes_median']),
                   xytext=(max_engagement['periodo_dt'], max_engagement['votes_median'] + 2),
                   arrowprops=dict(arrowstyle='->', color='purple', lw=2),
                   fontsize=10, ha='center', fontweight='bold',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="plum", alpha=0.8))
    
    # Leyenda consolidada
    lines1, labels1 = ax1_2.get_legend_handles_labels()
    lines2, labels2 = ax2_2.get_legend_handles_labels()
    lines3, labels3 = ax3_2.get_legend_handles_labels()
    ax1_2.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, 
                 loc='upper left', fontsize=11, framealpha=0.9)
    
    # Grid sutil
    ax1_2.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.savefig('informe_grafico2_rating_vader_engagement.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Gráficos para informe generados:")
    print("   📊 Gráfico 1: informe_grafico1_rating_cargador.png")
    print("   📊 Gráfico 2: informe_grafico2_rating_vader_engagement.png")
    print("   💡 Rating transparente en Gráfico 2 como contexto visual")

# Ejecutar generación de gráficos para informe
if 'triangulation_data' in locals() and triangulation_data is not None:
    print("🔄 Generando gráficos específicos para informe...")
    create_report_graphics(triangulation_data)
else:
    print("⚠️ Ejecuta primero la Sección 4 completa para generar triangulation_data")

In [ ]:
def create_report_graphics(triangulation_df):
    """
    Genera dos gráficos específicos para el informe:
    1. Rating + "Cargador" (mensaje central)
    2. Rating + VADER + Engagement (análisis profundo)
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    from matplotlib.dates import DateFormatter
    import matplotlib.dates as mdates
    
    # Configurar estilo para informe
    plt.style.use('default')
    sns.set_style("dark")
    
    # Colores por intensidad
    color_map = {
        'Normal': '#2E8B57',      # Verde
        'Baja': '#FFD700',        # Dorado  
        'Moderada': '#FF8C00',    # Naranja
        'Alta': '#FF4500',        # Naranja rojizo
        'Crítica': '#DC143C'      # Rojo
    }
    
    colors = [color_map.get(intensidad, '#808080') for intensidad in triangulation_df['intensidad']]
    
    # =============================================================================
    # GRÁFICO 1: RATING + "CARGADOR" (MENSAJE CENTRAL)
    # =============================================================================
    
    fig1, ax1 = plt.subplots(figsize=(12, 6))
    
    # Barras de rating con colores por intensidad
    bars1 = ax1.bar(triangulation_df['periodo_dt'], triangulation_df['rating_mean'], 
                    color=colors, alpha=0.6, width=25, edgecolor='black', linewidth=0.5)
    
    # Añadir valores en las barras
    for bar, rating in zip(bars1, triangulation_df['rating_mean']):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{rating:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    # Línea de frecuencia "cargador"
    ax2_1 = ax1.twinx()
    line_cargador = ax2_1.plot(triangulation_df['periodo_dt'], triangulation_df['freq_cargador'], 
                               'mo-', linewidth=4, markersize=10, label='Menciones "Cargador"', 
                               alpha=0.9, markeredgecolor='darkmagenta', markeredgewidth=1)
    
    # Configurar ejes
    ax1.set_ylabel('Rating Promedio', fontsize=14, fontweight='bold', color='black')
    ax2_1.set_ylabel('Menciones "Cargador" por Período', fontsize=14, fontweight='bold', color='darkred')
    ax1.set_xlabel('Período', fontsize=14, fontweight='bold')
    
    ax1.tick_params(axis='y', labelcolor='black', labelsize=12)
    ax2_1.tick_params(axis='y', labelcolor='darkred', labelsize=12)
    
    # Formatear fechas
    ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    ax1.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, fontsize=11)
    
    # Título y líneas de referencia
    ax1.set_title('Identificación Automática del Problema del Cargador\nCaso Samsung A15: Validación Cruzada de Técnicas', 
                  fontsize=16, fontweight='bold', pad=20)
    
    # Línea crítica en rating 3.0 (alineada a la izquierda)
    ax1.axhline(y=3.0, color='darkblue', linestyle=':', alpha=0.8, linewidth=2)
    ax1.text(ax1.get_xlim()[1], 3.1, 'Umbral Crítico', 
         fontsize=11, color='darkblue', fontweight='bold', ha='right', va='bottom')
    ax1.grid(False)
    
    # Leyenda (esquina superior derecha)
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2_1.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=12, framealpha=0.9)
    
    # Formato
    plt.tight_layout()
    plt.savefig('../outputs/visualizations/03_informe_grafico1_rating_cargador.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # =============================================================================
    # GRÁFICO 2: RATING + ANÁLISIS DE SENTIMIENTO + ENGAGEMENT (ANÁLISIS PROFUNDO)
    # =============================================================================
    
    fig2, ax1_2 = plt.subplots(figsize=(12, 6))
    
    # Barras de rating TRANSPARENTES como fondo/contexto
    bars2 = ax1_2.bar(triangulation_df['periodo_dt'], triangulation_df['rating_mean'], 
                      color=colors, alpha=0.25, width=25, 
                      edgecolor='gray', linewidth=0.8)
    
    # Línea de análisis de sentimiento (protagonista)
    ax2_2 = ax1_2.twinx()
    line_sentiment = ax2_2.plot(triangulation_df['periodo_dt'], triangulation_df['vader_sentiment'], 
                               'b^-', linewidth=4, markersize=10, label='Análisis de Sentimiento', 
                               alpha=0.9, markeredgecolor='darkblue', markeredgewidth=1)
    
    # Línea de engagement (protagonista)
    ax3_2 = ax1_2.twinx()
    ax3_2.spines['right'].set_position(('outward', 60))
    line_engagement = ax3_2.plot(triangulation_df['periodo_dt'], triangulation_df['votes_median'], 
                                 's--', color='purple', linewidth=3, markersize=8, 
                                 label='Engagement (Votos Útiles)', alpha=0.9,
                                 markeredgecolor='indigo', markeredgewidth=1)
    
    # Configurar ejes
    ax1_2.set_ylabel('Rating Promedio (referencia)', fontsize=13, fontweight='bold', color='darkgray')
    ax2_2.set_ylabel('Análisis de Sentimiento', fontsize=14, fontweight='bold', color='darkblue')
    ax3_2.set_ylabel('Engagement (Votos Útiles)', fontsize=14, fontweight='bold', color='purple')
    ax1_2.set_xlabel('Período', fontsize=14, fontweight='bold')
    
    ax1_2.tick_params(axis='y', labelcolor='gray', labelsize=11)
    ax2_2.tick_params(axis='y', labelcolor='darkblue', labelsize=12)
    ax3_2.tick_params(axis='y', labelcolor='purple', labelsize=12)
    
    # Formatear fechas
    ax1_2.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    ax1_2.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax1_2.xaxis.get_majorticklabels(), rotation=45, fontsize=11)
    
    # Título
    ax1_2.set_title('Análisis Emocional y de Respuesta Social\nCaso Samsung A15: Indicadores Multidimensionales', 
                    fontsize=16, fontweight='bold', pad=20)
    
    # Líneas de referencia (alineadas a la izquierda)
    ax2_2.axhline(y=0, color='darkblue', linestyle=':', alpha=0.6, linewidth=2)
    ax2_2.text(ax1_2.get_xlim()[1], 0.02, 'Sentimiento Neutral', 
           fontsize=11, color='darkblue', fontweight='bold', ha='right', va='bottom')
    
    # Zona crítica para sentimiento
    ax2_2.axhspan(-0.3, -0.05, alpha=0.1, color='red', label='Zona Negativa')
    
    # Leyenda consolidada (esquina superior derecha)
    lines1, labels1 = ax1_2.get_legend_handles_labels()
    lines2, labels2 = ax2_2.get_legend_handles_labels()
    lines3, labels3 = ax3_2.get_legend_handles_labels()
    ax1_2.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, 
                 loc='upper right', fontsize=11, framealpha=0.9)
    
   
    plt.tight_layout()
    plt.savefig('../outputs/visualizations/03_informe_grafico2_rating_sentiment_engagement.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Gráficos para informe generados:")
    print("   📊 Gráfico 1: informe_grafico1_rating_cargador.png")
    print("   📊 Gráfico 2: informe_grafico2_rating_sentiment_engagement.png")
    print("   💡 Versión adaptada para audiencia no técnica")

# Ejecutar generación de gráficos para informe
if 'triangulation_data' in locals() and triangulation_data is not None:
    print("🔄 Generando gráficos específicos para informe...")
    create_report_graphics(triangulation_data)
else:
    print("⚠️ Ejecuta primero la Sección 4 completa para generar triangulation_data")

# 🎯 Conclusiones: Síntesis de Resultados Metodológicos

## 📊 Resultados Principales del Caso de Estudio

### Convergencia Metodológica Exitosa

El análisis de triangulación EDA-NLP demostró una **convergencia metodológica del 80.0%** en el caso Samsung A15, validando la efectividad del framework para datasets pequeños.

**¿Qué significa convergencia metodológica?** Es el grado en que técnicas independientes (EDA cuantitativo, análisis semántico NLP, y vocabulario crítico) coinciden en identificar los mismos períodos como problemáticos. Una alta convergencia indica que múltiples enfoques analíticos están "viendo" el mismo fenómeno, aumentando la confianza en los hallazgos cuando el dataset es pequeño para validación estadística robusta.

**Resultados cuantitativos:**
- **Períodos analizados**: 10 (junio 2024 - marzo 2025)
- **Convergencias exitosas**: 8/10 períodos (≥2/3 criterios cumplidos)
- **Performance por criterio individual**:
  - VADER negativo (<-0.05): **90% detección** (9/10 períodos)
  - Vocabulario crítico "cargador" (≥5): **80% detección** (8/10 períodos) 
  - Rating crítico (≥Moderada): **70% detección** (7/10 períodos)

### Identificación Exitosa de Problema Específico

El framework identificó automáticamente el **problema del cargador** como causa central de la crisis Samsung A15:

- **Consistencia temporal**: Presente en 8/10 períodos analizados (80%)
- **Pico de menciones**: 31 referencias en octubre 2024
- **Persistencia**: Problema sostenido julio 2024 - marzo 2025
- **Especificidad**: Causa única y focalizada vs crisis multifactorial

## 🕐 Desfases Temporales Identificados

### NLP Como Indicador Adelantado Confirmado

El análisis reveló un **patrón consistente de desfase temporal** entre técnicas:

**Secuencia Temporal Observada:**
1. **Julio-Septiembre 2024**: Descenso abrupto VADER (0.300 → -0.221) anticipando crisis
2. **Octubre 2024**: Pico vocabulario crítico (31 menciones "cargador")
3. **Noviembre 2024**: Rating mínimo (2.70) + Score EDA máximo (4/4)
4. **Enero 2025**: VADER alcanza punto más negativo (-0.247) post-impacto

**Interpretación Metodológica:**
- **VADER anticipa** el deterioro desde julio, detectando el problema 3-4 meses antes del mínimo rating
- **Vocabulario crítico** identifica la causa específica 1 mes antes del impacto máximo
- **Rating confirma** el deterioro cuando ya es evidente en métricas cuantitativas
- **VADER continúa** profundizando emocionalmente incluso durante la recuperación inicial

### Valor del Desfase para Análisis Futuro

Este patrón sugiere que **el análisis semántico puede actuar como indicador adelantado** de deterioro reputacional, proporcionando ventana temporal para intervención antes de que el impacto se materialice completamente en ratings promedio.

## 📈 Lecciones Metodológicas Clave

### Fortalezas Demostradas del Framework

**1. Triangulación Robusta en Datasets Pequeños**
- 80% convergencia con solo 10 períodos analizados
- Múltiples técnicas independientes validan hallazgos
- Reduce dependencia de validación estadística tradicional

**2. Diagnóstico Específico Automatizado**
- Identificación de causas raíz sin análisis manual previo
- Vocabulario crítico emerge automáticamente del corpus
- Especificidad que supera métricas agregadas simples

## 🚀 Aplicabilidad y Replicabilidad

### Condiciones y Adaptaciones para Replicación

**Contextos Apropiados:**
- Datasets pequeños-medianos (5-50 períodos temporales)
- Productos con vocabulario crítico identificable
- Crisis focalizadas vs difusas multifactoriales

**Adaptaciones Requeridas:**
- Calibración de umbrales por mercado/contexto específico
- Selección de herramientas NLP apropiadas para idioma target
- Definición de vocabulario crítico relevante por industria

**Extensiones Metodológicas Propuestas:**
- Validación cruzada con múltiples crisis documentadas
- Sistema de alertas basado en umbral de convergencia
- Integración con fuentes de datos adicionales

## 💡 Contribuciones al Estado del Arte

### Valor Metodológico del Framework

**1. Triangulación en Contextos de Datos Limitados**
- Demuestra efectividad de validación cruzada con muestras pequeñas
- Principios transferibles a otras aplicaciones de análisis reputacional
- Framework flexible adaptable a diferentes contextos e industrias

**2. Integración Práctica EDA-NLP**
- Combina simplicidad interpretativa con sofisticación técnica
- Balance entre automatización y capacidad de diagnóstico específico
- Metodología escalable manteniendo principios fundamentales

---

**🎯 Síntesis Final:** Este caso de estudio demuestra que la triangulación metodológica puede extraer insights válidos y accionables de datasets pequeños, proporcionando tanto detección de anomalías como diagnóstico específico de causas, siempre que se reconozcan explícitamente las limitaciones contextuales y se diseñen expectativas realistas apropiadas para el alcance disponible.

In [ ]:
# =============================================================================
# GRÁFICO SIMPLE: RATING vs NLP (Sin Score EDA)
# =============================================================================

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def crear_grafico_rating_nlp_simple():
    """
    Gráfico directo: Rating absoluto vs Frecuencia NLP
    """
    
    # Configurar estilo darkgrid
    sns.set_style("darkgrid")
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['axes.facecolor'] = '#f0f0f0'
    
    # Datos Samsung EDA (solo necesitamos rating)
    samsung_eda = df_anomalias_eda[df_anomalias_eda['producto'] == 'Samsung A15'].copy()
    
    # Datos NLP (frecuencia cargador)
    samsung_nlp = [p for p in patrones_semanticos if p['producto'] == 'Samsung A15']
    df_nlp = pd.DataFrame(samsung_nlp)
    
    freq_cargador = []
    for _, row in df_nlp.iterrows():
        vocab = row['vocabulario_critico']
        freq = vocab.get('cargador', 0) if isinstance(vocab, dict) else 0
        freq_cargador.append(freq)
    
    df_nlp['freq_cargador'] = freq_cargador
    
    # Merge datos
    samsung_eda['periodo_str'] = samsung_eda['periodo'].astype(str)
    df_nlp['periodo_str'] = df_nlp['periodo'].astype(str)
    
    merged_simple = pd.merge(samsung_eda, df_nlp[['periodo_str', 'freq_cargador']], 
                            on='periodo_str', how='inner')
    merged_simple = merged_simple.sort_values('periodo').reset_index(drop=True)
    
    # Crear gráfico
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    # Eje izquierdo - Rating (INVERTIDO para mostrar crisis como picos hacia abajo)
    periodos_labels = [str(p)[-7:] for p in merged_simple['periodo']]
    x_pos = np.arange(len(periodos_labels))
    
    # Rating con colores según nivel crítico
    colors_rating = []
    for rating in merged_simple['rating_mean']:
        if rating < 2.8:
            colors_rating.append('#DC143C')  # Rojo - Crítico
        elif rating < 3.2:
            colors_rating.append('#FF8C00')  # Naranja - Severo
        elif rating < 3.6:
            colors_rating.append('#DAA520')  # Dorado - Moderado
        elif rating < 4.0:
            colors_rating.append('#32CD32')  # Verde claro - Aceptable
        else:
            colors_rating.append('#228B22')  # Verde oscuro - Bueno
    
    bars_rating = ax1.bar(x_pos, merged_simple['rating_mean'], 
                         alpha=0.8, color=colors_rating, width=0.6, 
                         edgecolor='black', linewidth=1.2)
    
    ax1.set_xlabel('Período', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Rating Promedio', fontsize=14, fontweight='bold', color='#1f4e79')
    ax1.tick_params(axis='y', labelcolor='#1f4e79', labelsize=12)
    ax1.set_ylim(1, 5)  # Escala completa de rating
    
    # Línea horizontal en rating crítico
    ax1.axhline(y=3.0, color='red', linestyle=':', alpha=0.7, linewidth=2)
    ax1.text(len(x_pos)-1, 3.1, 'Umbral Crítico (3.0)', ha='right', va='bottom', 
             fontsize=10, color='red', fontweight='bold')
    
    # Eje derecho - Frecuencia NLP y Mediana Votos Útiles
    ax2 = ax1.twinx()
    
    # Línea NLP
    line_nlp = ax2.plot(x_pos, merged_simple['freq_cargador'], 
                       color='#8B0000', marker='o', linewidth=4, 
                       markersize=12, markerfacecolor='#CD5C5C', 
                       markeredgecolor='#8B0000', markeredgewidth=2,
                       label='Frecuencia "cargador" (NLP)')
    
    # Línea Mediana Votos Útiles (Engagement)
    line_engagement = ax2.plot(x_pos, merged_simple['votes_median'], 
                              color='#4B0082', marker='s', linewidth=3, 
                              markersize=8, markerfacecolor='#9370DB', 
                              markeredgecolor='#4B0082', markeredgewidth=2,
                              linestyle='--', alpha=0.8,
                              label='Mediana Votos Útiles (Engagement)')
    
    ax2.set_ylabel('Frecuencia NLP / Mediana Votos Útiles', fontsize=14, fontweight='bold', color='#8B0000')
    ax2.tick_params(axis='y', labelcolor='#8B0000', labelsize=12)
    
    # Ajustar escala para acomodar ambas líneas
    max_freq = max(merged_simple['freq_cargador'])
    max_votes = max(merged_simple['votes_median'])
    ax2.set_ylim(0, max(max_freq, max_votes) + 5)
    
    # Configurar x-axis
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(periodos_labels, rotation=45, ha='right', fontsize=12)
    
    # Destacar convergencias: Rating bajo + NLP alto + Engagement alto
    for i, (rating, freq, engagement) in enumerate(zip(merged_simple['rating_mean'], 
                                                       merged_simple['freq_cargador'],
                                                       merged_simple['votes_median'])):
        # Marcar períodos de triple convergencia
        if rating < 3.5 and freq > 10 and engagement > 5:
            ax1.axvline(x=i, color='purple', linestyle=':', alpha=0.6, linewidth=2)
            ax1.text(i, 4.8, '⚠️', ha='center', fontsize=14)
    
    # Destacar desfase temporal: Pico NLP (Oct) vs Mínimo Rating (Nov)
    pico_nlp_idx = merged_simple['freq_cargador'].idxmax()
    min_rating_idx = merged_simple['rating_mean'].idxmin()
    pico_engagement_idx = merged_simple['votes_median'].idxmax()
    
    # Marcar pico NLP
    ax2.annotate('Pico Discusión\n"Cargador"', 
                xy=(pico_nlp_idx, merged_simple.iloc[pico_nlp_idx]['freq_cargador']),
                xytext=(pico_nlp_idx-0.8, merged_simple.iloc[pico_nlp_idx]['freq_cargador']+8),
                fontsize=10, fontweight='bold', color='#8B0000',
                arrowprops=dict(arrowstyle='->', color='#8B0000', lw=2),
                ha='center', bbox=dict(boxstyle="round,pad=0.3", facecolor='#FFE4E1'))
    
    # Marcar mínimo rating
    ax1.annotate('Mínimo\nRating', 
                xy=(min_rating_idx, merged_simple.iloc[min_rating_idx]['rating_mean']),
                xytext=(min_rating_idx+0.8, merged_simple.iloc[min_rating_idx]['rating_mean']-0.4),
                fontsize=10, fontweight='bold', color='#DC143C',
                arrowprops=dict(arrowstyle='->', color='#DC143C', lw=2),
                ha='center', bbox=dict(boxstyle="round,pad=0.3", facecolor='#FFE4E1'))
    
    # Marcar pico engagement si es diferente
    if pico_engagement_idx != pico_nlp_idx:
        ax2.annotate('Pico\nEngagement', 
                    xy=(pico_engagement_idx, merged_simple.iloc[pico_engagement_idx]['votes_median']),
                    xytext=(pico_engagement_idx+0.3, merged_simple.iloc[pico_engagement_idx]['votes_median']+3),
                    fontsize=9, fontweight='bold', color='#4B0082',
                    arrowprops=dict(arrowstyle='->', color='#4B0082', lw=1.5),
                    ha='center', bbox=dict(boxstyle="round,pad=0.2", facecolor='#E6E6FA'))
    
    # Flecha mostrando desfase temporal
    if pico_nlp_idx != min_rating_idx:
        ax1.annotate('', xy=(min_rating_idx, 4.5), xytext=(pico_nlp_idx, 4.5),
                    arrowprops=dict(arrowstyle='<->', color='purple', lw=3))
        ax1.text((pico_nlp_idx + min_rating_idx)/2, 4.7, 'Desfase\nTemporal', 
                ha='center', va='bottom', fontsize=10, fontweight='bold', color='purple')
    
    # Añadir valores en puntos clave
    for i, (rating, freq) in enumerate(zip(merged_simple['rating_mean'], merged_simple['freq_cargador'])):
        # Mostrar valores en picos/mínimos
        if i == pico_nlp_idx or i == min_rating_idx:
            ax1.text(i, rating + 0.1, f'{rating:.2f}', ha='center', va='bottom', 
                    fontweight='bold', fontsize=10, color='#1f4e79')
            ax2.text(i, freq + 1, f'{int(freq)}', ha='center', va='bottom', 
                    fontweight='bold', fontsize=10, color='#8B0000')
    
    # Título
    fig.suptitle('Rating vs Discusión "Cargador" vs Engagement: Caso Samsung A15', 
                fontsize=16, fontweight='bold', y=0.96)
    ax1.set_title('Análisis de desfases temporales entre rating, discusión semántica y engagement', 
                 fontsize=12, style='italic', pad=20, color='#555555')
    
    # Leyenda combinada
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#228B22', label='Rating > 4.0: Excelente'),
        Patch(facecolor='#32CD32', label='Rating 3.6-4.0: Bueno'),
        Patch(facecolor='#DAA520', label='Rating 3.2-3.6: Moderado'),
        Patch(facecolor='#FF8C00', label='Rating 2.8-3.2: Bajo'),
        Patch(facecolor='#DC143C', label='Rating < 2.8: Crítico'),
        plt.Line2D([0], [0], color='#8B0000', marker='o', markersize=8, 
                  markerfacecolor='#CD5C5C', label='Frecuencia "cargador" (NLP)'),
        plt.Line2D([0], [0], color='#4B0082', marker='s', markersize=6, 
                  markerfacecolor='#9370DB', linestyle='--', label='Mediana Votos Útiles')
    ]
    
    ax1.legend(handles=legend_elements, loc='center left', framealpha=0.95, 
              fontsize=10, fancybox=True, shadow=True)
    
    # Grid
    ax1.grid(True, alpha=0.6, color='#888888', linewidth=0.8)
    ax2.grid(True, alpha=0.3, color='#CCCCCC', linewidth=0.5)
    
    plt.tight_layout()
    
    # Guardar
    plt.savefig('rating_vs_nlp_desfase.png', dpi=300, bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    plt.show()
    
    return merged_simple

# Ejecutar
print("📊 Generando gráfico simple Rating vs NLP...")
datos_rating_nlp = crear_grafico_rating_nlp_simple()

# Análisis del desfase temporal
pico_nlp = datos_rating_nlp['freq_cargador'].idxmax()
pico_engagement = datos_rating_nlp['votes_median'].idxmax()
min_rating = datos_rating_nlp['rating_mean'].idxmin()

print(f"\n🕐 ANÁLISIS DE DESFASES TEMPORALES:")
print("="*60)
print(f"Pico NLP (máxima freq 'cargador'): {datos_rating_nlp.iloc[pico_nlp]['periodo']} "
      f"({datos_rating_nlp.iloc[pico_nlp]['freq_cargador']} menciones)")
print(f"Pico Engagement (mediana votos): {datos_rating_nlp.iloc[pico_engagement]['periodo']} "
      f"({datos_rating_nlp.iloc[pico_engagement]['votes_median']} votos)")
print(f"Mínimo Rating: {datos_rating_nlp.iloc[min_rating]['periodo']} "
      f"({datos_rating_nlp.iloc[min_rating]['rating_mean']:.2f} estrellas)")

print(f"\n📈 INTERPRETACIÓN TRIANGULAR:")
print(f"• Discusión 'cargador' máxima: {datos_rating_nlp.iloc[pico_nlp]['periodo']}")
print(f"• Engagement máximo: {datos_rating_nlp.iloc[pico_engagement]['periodo']}")  
print(f"• Impacto rating mínimo: {datos_rating_nlp.iloc[min_rating]['periodo']}")
print(f"• Patrón: Discusión → Engagement → Impacto en Rating")

# Análisis adicional de convergencias
print(f"\n🎯 PERÍODOS DE CONVERGENCIA MÚLTIPLE:")
for i, row in datos_rating_nlp.iterrows():
    if row['rating_mean'] < 3.5 and row['freq_cargador'] > 10 and row['votes_median'] > 5:
        print(f"   • {row['periodo']}: Rating {row['rating_mean']:.2f}, "
              f"NLP {row['freq_cargador']}, Engagement {row['votes_median']}")

print("\n✅ Gráfico guardado como 'rating_vs_nlp_desfase.png'")